[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MatteoFalcioni/Learning-LangGraph/blob/main/notebooks/10_deep_agents.ipynb)

### Setup

#### Install requirements

In [ ]:
%pip install -q -U -r https://raw.githubusercontent.com/MatteoFalcioni/Learning-LangGraph/main/requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### local (notebooks or files)

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # load api keys

True

#### Colab

In [ ]:
from google.colab import userdata
import os

REQUIRED_KEYS = [
    'OPENAI_API_KEY',
    'LANGSMITH_TRACING',
    'LANGSMITH_ENDPOINT',
    'LANGSMITH_API_KEY',
    'LANGSMITH_PROJECT'
]

def _set_colab_keys(key : str):
    # Retrieve the secret value using its key/name
    secret_value = userdata.get(key)
    # set it as a standard OS environment variable
    os.environ[key] = secret_value

for key in REQUIRED_KEYS:
    _set_colab_keys(key)

# Deep Agents & Middleware

In this notebook we will og over two concepts that are very new in LangGraph (at least in december 2025 while I'm writing this): Deep Agents and Middleware. 

Deep agents, also referred to sometimes as "agent harnesses" are a particular kind of agents that can go "in depth" into their workflow: we'll see what this actually means in a second. 

Middleware, on the other hand, is not a LangGraph concept: it's a term used to refer to all software layers that "glue together" different applications, databases or services. 

We'll see how LangChain developers translated this idea into some really useful LangGraph implementations. 

## 1. Deep Agents

Citing LangChain's [Deep Agents main page](https://docs.langchain.com/oss/python/deepagents/overview): 

*deepagents is a standalone library for building agents that can tackle complex, multi-step tasks. Built on LangGraph and inspired by applications like Claude Code, Deep Research, and Manus, deep agents come with planning capabilities, file systems for context management, and the ability to spawn subagents* 

*Use deep agents when you need agents that can:*

- *Handle complex, multi-step tasks that require planning and decomposition*
- *Manage large amounts of context through file system tools*
- *Delegate work to specialized subagents for context isolation*
- *Persist memory across conversations and threads*

**For simpler use cases, consider using LangChain’s create_agent or building a custom LangGraph workflow.**

This above ^ is highlighted because I think it's a concept that we cannot let go over our heads: deep agents are really powerful, but we will see that they are "heavyweight": for more simple tasks, there is no need to use this heavy machinery. 

### 1.1 Architecture

[This very interesting blog post](https://blog.langchain.com/deep-agents/) from LangChain explains perfectly the architecture of deep agents. I encouracge you to read it, or if you are more of a yt enjoyer, here's the [Deep Agents video by LangChain](https://youtu.be/IVts6ztrkFg?si=o51HELlDVN3YR6Ei).

The core algorithm is actually the same - it’s an LLM running in a loop calling tools. The difference compared to the naive agent that is easy to build is:

- A detailed system prompt
- Planning tool
- Sub agents
- File system

Now, in order to go over some deep agents application, we will not go into the specific details of the architecture. However, you can find a detailed youtube video from LangChain devs explaining the specific details here: [Rewriting Deep Agents on top of LangChain 1.0](https://youtu.be/AZ6257Ya_70?si=QSw20KdbgUixCb2p).

Here you can find the previous implementation, which is slightly updated but goes in depth into the actual structure of deep agents: [Implementing deepagents: a technical walkthrough](https://youtu.be/TTMYJAw5tiA?si=sCRI95ihge_1n_qu). I find it really interesting.

There is also a LangChain academy course specifically on deep agents: [LangChain Academy - Deep Agents](https://academy.langchain.com/courses/deep-agents-with-langgraph/?utm_medium=social&utm_source=youtube&utm_campaign=q3-2025_deep-agents-course_co).

Now, let's quickly go over the cited components:

<center>
<img src="images/deep_agents.png" width="1000">
<center>

#### 1.1.1 Planning Tool 

Claude Code uses a Todo list tool, and this is the inspiration for the todo tool that comes by default with a deep agent. You will be amazed of how useful this can be for your agents. 

#### 1.1.2 Spawning Subagents 

Deep agents go deeper on topics. This is largely accomplished by spinning up sub agents that specifically focused on individual tasks, and allowing those sub agents to go deep there.

Basically your agent will be creating subagents and assigning them tasks. After the subagent complete their tasks, they report back to the main agent.

#### 1.1.3 Virtual File System

Deep agents run for long periods of time and accumulate a lot of context that they need to manage. LangChain equipped Deep Agents with a file system tool, which creates a virtual file system in the agent's state, that the agent can use to write out notes, in order to manage context. 

These files are not created in your machine, they are just dictionaries in the agent's state.

#### 1.1.4 System Prompt

We all know by now how important a prompt is. Deep agents have longer context (like Claude Code) and therefore their prompts can be huge. If we want to write our own instructions, these will be added to the pre-built deep agents system prompt that LangChain devs built.

## 1.2 Example

Since our course has a practical setup, we will dive right in with an example.

We are going to build a Deep Research Agent that searches the web and writes report on a specific topic, following this nice LangChain yt video: [Build a Research Agent with Deep Agents](https://youtu.be/5tn6O0uXYEg?si=pHKBOz6zYstytPqS) (GitHub repo here: [Deep Agents Quickstart](https://github.com/langchain-ai/deepagents-quickstarts/tree/main)).

Here comes that time of the course where notebooks are starting to feel limited and messy for our applications. That is why in this lecture we will start deferring to the `projects/` directory: specifically, the example in question can be found in [projects/deep_agents](../projects/deep_agents/) directory. 

You can easily follow in the notebook [research_agent.ipynb](../projects/deep_agents/deep_research/research_agent.ipynb).

## 2. Middleware

Now let's quickly go over middleware. It's super simple to use.

Middleware runs hooks before and/or after model execution, in this way: 

<img src="./images/middleware.png" width=400>

LangChain provides prebuilt middleware for common use cases. Each middleware is production-ready and configurable:
https://docs.langchain.com/oss/python/langchain/middleware/built-in#provider-agnostic-middleware

Some middleware add tools to your agent, some add prompts, some add additional functionalities like prompt caching or tool retries, and there is also a human in the loop middleware. You have a wide choice.

## 2.1 Human In The Loop Middleware

Since we recently saw HITL, let's see how we can implement human in the loop before tool calls with middleware with two lines of code.

(Recall that HITL requires a checkpointer)

In [1]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver


def read_email_tool(email_id: str) -> str:
    """read an email by its ID."""
    return f"Email content for ID: {email_id}"

def send_email_tool(recipient: str, subject: str, body: str) -> str:
    """send an email."""
    return f"Email sent to {recipient} with subject '{subject}'"

agent = create_agent(
    model="gpt-4o-mini",
    tools=[read_email_tool, send_email_tool],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool": {
                    "allowed_decisions": ["approve", "edit", "reject"],
                },
                "read_email_tool": False,  # do not interrupt on this 
            }
        ),
    ],
)

In [2]:
from langchain_core.messages import HumanMessage

config = {"configurable" : {"thread_id" : "test"}}

result = agent.invoke(
        {
            "messages" : [HumanMessage(content="Send Matteo an email asking him if he comes to work tomorrow. No need to make it formal.")]
        },
        config=config
    )

In [3]:
interrupt = result['__interrupt__'][0]
for key, value in interrupt.value.items():
    print(f"{key} : {value}")

action_requests : [{'name': 'send_email_tool', 'args': {'recipient': 'Matteo', 'subject': 'Work Tomorrow?', 'body': 'Hey Matteo, are you coming to work tomorrow?'}, 'description': "Tool execution requires approval\n\nTool: send_email_tool\nArgs: {'recipient': 'Matteo', 'subject': 'Work Tomorrow?', 'body': 'Hey Matteo, are you coming to work tomorrow?'}"}]
review_configs : [{'action_name': 'send_email_tool', 'allowed_decisions': ['approve', 'edit', 'reject']}]


In [4]:
from langgraph.types import Command

agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

{'messages': [HumanMessage(content='Send Matteo an email asking him if he comes to work tomorrow. No need to make it formal.', additional_kwargs={}, response_metadata={}, id='c7dad376-334e-4caf-b373-3db72b5982a9'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 93, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cxy5Sm36yv2BybdkkMh2fJ0AqUwNK', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bbd5a-2871-7d52-9d2f-ced4b9bff7c1-0', tool_calls=[{'name': 'send_email_tool', 'args': {'recipient': 'Matteo', 'subject': 'Work Tomorrow?', 'body': 'Hey Matteo, are you c

## 2.2 To Do List 

Next, one of the most simple and at the same time useful tools implemented in middleware: we can allow our agent to write and update a todo list to keep track of its progresses. You'll be suprised of how better your agent performs when using this tool.

This middleware automatically provides agents with a write_todos tool and system prompts to guide effective task planning.

```python
from langchain.agents import create_agent
from langchain.agents.middleware import TodoListMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[read_file, write_file, run_tests],  # example tools
    middleware=[TodoListMiddleware()],
)
```

## 2.3 File Search (actual files in your folder)

This is a cool one: look for files in your file system

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import FilesystemFileSearchMiddleware
from langchain.messages import HumanMessage

agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        FilesystemFileSearchMiddleware(
            root_path="./",
            use_ripgrep=True,
            max_file_size_mb=10,
        ),
    ],
)

# Agent can now use glob_search and grep_search tools
result = agent.invoke({
    "messages": [HumanMessage("Find all jupyter notebooks containing @tool decorators")]
})

In [6]:
print(result['messages'][-1])

content='The following Jupyter notebooks contain `@tool` decorators:\n\n1. `/10_deep_agents.ipynb`\n2. `/11_supervisor_&_MCP.ipynb`\n3. `/12_hierarchical_agent_teams.ipynb`\n4. `/13_voice_agents.ipynb`\n5. `/15_RAG_pt2.ipynb`\n6. `/3.5_recap.ipynb`\n7. `/3_llms_&_agents.ipynb`\n8. `/5_command.ipynb`\n9. `/6_agentic_graph.ipynb`\n10. `/7.5_recap.ipynb`\n11. `/7_memory.ipynb`\n12. `/8_human_in_the_loop.ipynb`\n13. `/9_agent_collaboration.ipynb`' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 164, 'prompt_tokens': 556, 'total_tokens': 720, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a0e9480a2f', 'id': 'chatcmpl-Cxy6XtA93sBrxExdA3W4pUiawBPcI', 'service_tier': 'default', 'finish_reason': 'stop', 'log

## 2.4 Shell Tool

Expose a persistent shell session to agents for command execution. Shell tool middleware is useful for the following:

- Agents that need to execute system commands
- Development and deployment automation tasks
- Testing and validation workflows
- File system operations and script execution

Use appropriate execution policies (HostExecutionPolicy, DockerExecutionPolicy, or CodexSandboxExecutionPolicy) to match your deployment’s security requirements.

In [7]:
from langchain.agents import create_agent
from langchain.agents.middleware import (
    ShellToolMiddleware,
    HostExecutionPolicy,
)

agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        ShellToolMiddleware(
            workspace_root="./",
            execution_policy=HostExecutionPolicy(),  # agent can do rm -rf * be careful
        ),
    ],
)

In [8]:
result = agent.invoke({
    "messages": [HumanMessage("Create a new folder named tests/")]
})

In [9]:
for message in result['messages']:
    print(message.content)

Create a new folder named tests/

<no output>
The folder named `tests` has been successfully created.
